### Notebook content

Step by step
- load data
- extract text
- store data ( paper id and text document) in data_dictionary

Sintesis of the above things(final code v1)
- load data
- extract and store data
- assert results

### Notebook content 1

Api bioportal
- api conection
- query
- results
- query resources bioportal: all ontologies

In [1]:
import os
import time
import json
import urllib
from urllib import parse, request
import pandas as pd

###  Step by step

#### Load data.
For this build path for every file in directory. paths store in a list. from the list of paths each file is read.


In [ ]:
pdf_json = []
for root, dirs, files in os.walk('kaggle/document_parses/pdf_json/'): #Case:files under  ../../pdf_json directory
    for file in files:
        pdf_json.append(os.path.join(root, file)) #build a path for each file

In [ ]:
pdf_json # list of paths 

#### Extract data firts record:  

In [ ]:
with open(pdf_json[0]) as json_data:
    data = json.load(json_data)

data.keys() # metadata record

In [ ]:
# data['metadata']['title']

data['metadata']['title'] # title record. firt record not have a title

In [ ]:

# data['abstract']

data['abstract'] #neested data structure: list thar have inside dictionaries. each dictionary is a paragraph.

content_abstract = ''
for paragraph in data['abstract']:
    content_abstract += paragraph['text']

data=content_abstract

In [ ]:
# data['body_text']

data['body_text'] #neested data structure again: list that have inside dictionaries. each dictionary is a paragraph.

len(data['body_text']) # have 37 paragrphs

In [ ]:
content_body = ''
for paragraph in data['body_text']:
    content_body += paragraph['text']

content_body

In [ ]:
#### document_text = content_abstract + content_body

document_text =  data['metadata']['title'] + content_abstract + content_body # total content firts record

print(document_text)

####  Store data in a data dictionary(schema db proposal)

data_dict = {'paper_id':[], 'documents_texts':[], 'bioportal_anotations':[]}

#### Load paper id y document en el data_dict para el primer registro

In [ ]:
data_dict['paper_id'].append(data['paper_id'])
data_dict['documents_texts'].append(document_text)

In [ ]:
data_dict

### Sintesis of the above things

In [26]:
#v.1.1 load and store text data to anotate

path_data = []
for root, dirs, files in os.walk('kaggle/document_parses/'): 
    for file in files:
        path_data.append(os.path.join(root,file))

#extract content of the all records. load into data_dictionari
data_dict = {'paper_idx':[], 'paper_id':[],'title':[],
             'content_to_annotate':[], 'document_text':[],
             'bioportal_anotations':[]}

idx = 0
for path in path_data:
    
    with open(path, 'r') as json_data:
        data = json.load(json_data)
        
    
    try:              #for pdf_json
        content_abstract = ''
        for paragraph in data['abstract']:
            content_abstract += ' ' + paragraph['text']
            
        
        content_body = ''
        for paragraph in data['body_text']:
            content_body += ' ' + paragraph['text']
        
        document_text = data['metadata']['title'] + content_abstract + content_body 
        
        content_to_annotate = data['metadata']['title'] + ' ' + content_abstract

                    
    except KeyError:    # for pmc_json
        
        content_to_annotate= data['metadata']['title'] + ' ' + data['body_text'][0]['text']
        content_body = ''
        for paragraph in data['body_text']:
            content_body += ' ' + paragraph['text']

        document_text = data['metadata']['title'] + content_body
    
    data_dict['paper_idx'].append(idx)
    data_dict['paper_id'].append(data['paper_id'])        
    data_dict['title'].append(data['metadata']['title'])
    data_dict['content_to_annotate'].append(content_to_annotate)
    data_dict['document_text'].append(document_text)
    idx += 1

In [3]:
#function load text in data dict v.1
def load_text_for_paper_in_directory(path_to_directory):
    '''v.1.1 load and store text data to anotate
    
    args:
    process:
    output:
    
    '''
    
    global path_data
    path_data = []
    for root, dirs, files in os.walk(path_to_directory): 
        for file in files:
            path_data.append(os.path.join(root,file))

    #extract content of the all records. load into data_dictionari
    global data_dict
    data_dict = {'paper_idx':[], 'paper_id':[],'title':[],
                 'content_to_annotate':[], 'document_text':[],
                 'bioportal_anotations':[]}

    idx = 0
    for path in path_data:
        
        with open(path, 'r') as json_data:
            data = json.load(json_data)

        try:              #for pdf_json
            content_abstract = ''
            for paragraph in data['abstract']:
                content_abstract += ' ' + paragraph['text']


            content_body = ''
            for paragraph in data['body_text']:
                content_body += ' ' + paragraph['text']

            document_text = data['metadata']['title'] + content_abstract + content_body 

            content_to_annotate = data['metadata']['title'] + ' ' + content_abstract


        except KeyError:    # for pmc_json

            content_to_annotate= data['metadata']['title'] + ' ' + data['body_text'][0]['text']
            content_body = ''
            for paragraph in data['body_text']:
                content_body += ' ' + paragraph['text']

            document_text = data['metadata']['title'] + content_body

        data_dict['paper_idx'].append(idx)
        data_dict['paper_id'].append(data['paper_id'])        
        data_dict['title'].append(data['metadata']['title'])
        data_dict['content_to_annotate'].append(content_to_annotate)
        data_dict['document_text'].append(document_text)
        idx += 1
    return data_dict

In [28]:
#if the ejecution of this line not raise anything, every its good 
assert len(path_data) == len(data_dict['paper_id']) and len(path_data) == len(data_dict['document_text']), 'wrong'

### Bioportal apy query 

In [31]:
REST_URL = "http://data.bioontology.org"
API_KEY = "3b00793b-f3cc-489c-9d6e-7a888f4b656e"
ontologies = "ontology1,ontology2,..."

In [32]:
### list of ontologies bioportal

def get_json(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    return json.loads(opener.open(url).read())

resources = get_json(REST_URL + "/")

ontologies = get_json(resources["links"]["ontologies"])

ontology_output = []
for ontology in ontologies:
    ontology_output.append(f"{ontology['name']}\n{ontology['@id']}\n")

#for ont in ontology_output:
    #print(ont)

In [37]:
### Ontologies selected

ont_sel = pd.read_csv('select_onts.csv')

list_onto = [x.replace('\t','').replace(' ','').replace('\n','') for x in list(ont_sel['Ontology'].values) if type(x)!=float]

list_uniq = []
for i in list_onto:
    if i not in list_uniq:
        list_uniq.append(i)
    else:
        next

#list_uniq

In [38]:
# merge ontologies bioportal with ontologies selected
ontologies = ' ' 
for onto in ontology_output:
    
    for i in list_uniq:
        
        if onto.find(i)>0:
            if ontologies.find(i) > 0:
                next
            else:
                ontologies += i + ' '     
ontologies = ontologies[1:-1].replace(' ',',') #param query

In [39]:
#make url query function
def make_url_query(API_KEY,
                   ontologies, 
                   expand_class_hierachy_text,
                   class_hierachy_max_level, text):
    global url
    url = REST_URL+ """/annotator/?apikey={}&ontologies={}&expand_class_hierachy={}&class_hierachy_max_level={}&text={}""".format(API_KEY, ontologies,
                                          expand_class_hierachy_text,class_hierachy_max_level,text)
    return url

In [51]:
#v.1 api calls
def make_query_bioportal_api(index_data_dict, 
                             API_KEY,
                             ontologies,
                             expand_class_hierachy_text,
                             class_hierachy_max_level):
    
    for i in index_data_dict:                    #data_dict['paper_idx']:
            
        #text to anotate extract from data_dict
        text = parse.quote(data_dict['content_to_annotate'][i])
        #make url
            
        make_url_query(API_KEY, ontologies, expand_class_hierachy_text, class_hierachy_max_level, text)
        #call api
        api_call = request.urlopen(url)
        annotations = json.loads(api_call.read().decode('utf-8'))

        #store anotations
        data_dict['bioportal_anotations'].append(annotations)
    return data_dict

In [53]:
make_query_bioportal_api(data_dict['paper_idx'][:4], API_KEY, ontologies, 'true',3)

{'paper_idx': [],
 'paper_id': [],
 'title': [],
 'content_to_annotate': [],
 'document_text': [],
 'bioportal_anotations': []}